In [1]:
import collections
import time
import numpy as np
import grpc

import tensorflow as tf
import tensorflow_federated as tff

import nest_asyncio
nest_asyncio.apply()

In [2]:
@tff.tf_computation(tf.int64)
def make_data(n):
   return tf.data.Dataset.range(n)

@tff.tf_computation(tff.SequenceType(tf.int64))
def sum_data(dataset):
   return dataset.reduce(np.int64(0), lambda x, y: x + y)

@tff.federated_computation(tff.FederatedType(tf.int64, tff.CLIENTS))
def compute_local_sum(federated_n):
    dataset   = tff.federated_map(make_data, federated_n)
    local_sum = tff.federated_map(sum_data, dataset)
    return local_sum

@tff.federated_computation(tff.FederatedType(tf.int64, tff.CLIENTS))
def compute_federated_sum(federated_n):
    return tff.federated_sum(compute_local_sum(federated_n))

In [3]:
ip_address = '0.0.0.0'  #@param {type:"string"}
port = 8000  #@param {type:"integer"}

channels = [grpc.insecure_channel(f'{ip_address}:{port}') for _ in range(1)]

tff.backends.native.set_remote_execution_context(channels, rpc_mode='REQUEST_REPLY')

In [4]:
channels

In [5]:
#tff.backends.native.set_local_execution_context()

In [ ]:
compute_federated_sum([5])